# 🦜LangChain `create_agent`

**Welcome!**
In the previous notebook, we built a graph manually (nodes, edges, state) using LangGraph. That gave us total control but required a lot of setup.
This notebook introduces **`create_agent`**, a high-level function from the `langchain` module.

###  What is `create_agent`?
It is a **pre-assembled LangGraph**.
Under the hood, `create_agent` automatically:
1.  Creates a `StateGraph`.
2.  Adds a model node and a tool node.
3.  Connects them with the correct edges (loops).
4.  Compiles it into a `CompiledGraph`.

### **📚 What We Will Cover**
1.  **`create_agent`:** Instantiating a pre-built ReAct agent workflow (Model + Tools loop) without needing to define nodes and edges manually.
2.  **Standard Interrupts:** Handling missing info (just like before).
3.  **Middleware:** A way to hook into the agent's brain to modify its behavior (e.g., adding a "Human Approval" step before dangerous tools are run).

In [1]:
from langchain.agents import create_agent
from langraph_prompts import ACTION_PROMPT_TEMPLATE
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command, interrupt
from datetime import datetime

In [2]:
@tool
def get_time_off_balance(user_id: str) -> int:
    '''
    Get the time off balance for a user. 

    Args :
        user_id: A unique string representing the user id

    Returns:
        An integer value representing the time-off balance
    '''
    return 10

@tool
def process_time_off(user_id: str, start_date: str, end_date: str) -> dict:
    '''
    Process time-off request for a user

    Args :
        user_id: A unique string representing the user id
        start_date: A date string in YYYY-MM-DD format for the start date of the time-off request
        end_date: A date string in YYYY-MM-DD format for the end date of the time-off request

    Returns:
        A dictionary with the following keys:
        - status: A string value representing the status of the time-off request
        - message: A string value representing the message of the time-off request
    '''
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    days_requested = end_date - start_date
    time_off_balance = get_time_off_balance.invoke({"user_id": user_id})

    result = {}

    if days_requested.days + 1 > time_off_balance:
        result['status'] = 'error'
        result['message'] = f'Time off request failed. you only have {time_off_balance} days of remaining leaves but requested {days_requested.days} days'

    else:
        time_off_balance -= days_requested.days + 1
        result['status'] = 'success'
        result['message'] = f'Time off request processed successfully for {days_requested.days + 1} days. Your remaining time off balance is {time_off_balance}'


    return result


@tool
def get_additional_info_from_human( message: str) -> str:
    '''
        Raises an interrupt to fetch additional information from the human requesting the action

        Args:
            message: a message string from the AI requesting the user for missing information

        Returns:
            A string value representing the response with additional information from the human

    '''
    interrupt_result = interrupt(message)

    return interrupt_result['user_message']


## 1) Creating the Agent (The Easy Way)

* **Manual Graph (Previous Notebook):** Like building a PC from scratch. You buy the CPU, RAM, and Motherboard, and connect the wires yourself. It's powerful but takes time.
* **`create_agent` (This Notebook):** Like buying a pre-built laptop. You just turn it on. It has the same components inside (LLM, Tools, Memory), but LangChain assembled them for you.

###  Code Context
In the code below:
* `create_agent(...)`: This single function replaces the `StateGraph`, `add_node`, `add_edge`, and `compile` steps.
* It automatically creates a graph where the LLM decides to call tools and loops back until it's done.
* `checkpointer=InMemorySaver()`: We still need this to enable "Pause/Resume" functionality.

📚 **[Docs: create_agent](https://docs.langchain.com/oss/python/langchain/agents)**

In [3]:
user_id = '1' 
llm = init_chat_model(model="gpt-4o", temperature=0)
tools = [get_time_off_balance, process_time_off, get_additional_info_from_human]

todays_date = datetime.now().strftime("%Y-%m-%d")

system_prompt = PromptTemplate(template=ACTION_PROMPT_TEMPLATE).invoke({'todays_date': todays_date, 'user_id': user_id})

agent = create_agent(llm, tools=tools, system_prompt = system_prompt.text, checkpointer=InMemorySaver())

In [4]:
user_message = "I want to take leave"

config = {'configurable': {'thread_id': user_id}}

In [5]:
result = agent.invoke({"messages": [{"role": "user", "content": user_message}]}, config=config)

result

{'messages': [HumanMessage(content='I want to take leave', additional_kwargs={}, response_metadata={}, id='c355c23c-c91b-402d-a7e6-94c690d1a38f'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 343, 'total_tokens': 373, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e1dc348028', 'id': 'chatcmpl-CnivJGhc2gsHzYnvc3ln0qKjlBpnn', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d8947638-b088-48f5-989a-a121c0fe47a9-0', tool_calls=[{'name': 'get_additional_info_from_human', 'args': {'message': 'Please provide the start and end dates for your leave request.'}, 'id': 'call_yMFLQx6Cjxz6NGOwfwOPk6G3', 'type': 'tool_call'}],

## 1.1) Resuming the Agent

Just like in the previous notebook, our `get_additional_info_from_human` tool triggered an **Interrupt**. The agent is now paused, waiting for the "End Date".
We use the same `Command(resume=...)` pattern to provide the answer and un-pause execution.

### Code Context
In the code below:
* `agent.invoke(...)`: We pass the `Command` object with the user's answer (`"tomorrow"`).
* The agent takes this answer, pretends the tool returned it, and continues its logic to book the leave.

In [6]:
user_message = "tomorrow"

result = agent.invoke(Command(resume={'user_message': user_message}), config=config)

result

{'messages': [HumanMessage(content='I want to take leave', additional_kwargs={}, response_metadata={}, id='c355c23c-c91b-402d-a7e6-94c690d1a38f'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 343, 'total_tokens': 373, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e1dc348028', 'id': 'chatcmpl-CnivJGhc2gsHzYnvc3ln0qKjlBpnn', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d8947638-b088-48f5-989a-a121c0fe47a9-0', tool_calls=[{'name': 'get_additional_info_from_human', 'args': {'message': 'Please provide the start and end dates for your leave request.'}, 'id': 'call_yMFLQx6Cjxz6NGOwfwOPk6G3', 'type': 'tool_call'}],

## 3) Middleware (Human-in-the-Loop)

Middleware in LangChain is a new abstraction that lets you hook into and control the agent’s core loop at defined points (before the model is called, as the model is called, and after the model/tool step finishes). It is conceptually similar to HTTP middleware in web frameworks: you can stack multiple middleware components that each inspect or modify state as a request “flows” through the agent.

Each middleware can read and modify the agent state, messages, tools, or model configuration at specific hook points, without changing the agent implementation itself.

In our code, we are adding this "Are you sure?" popup to the **Book Leave** tool to prevent accidental bookings.

### Code Context
In the code below:
* `HumanInTheLoopMiddleware`: This is the component that intercepts tool calls.
* `interrupt_on`: This dictionary defines **which** tools trigger the pause. We map `process_time_off` to specific permissions, ensuring the agent cannot run this specific tool without external approval.

📚 **[Docs: Middleware](https://docs.langchain.com/oss/python/langchain/middleware/overview)**

In [13]:
from langchain.agents.middleware import HumanInTheLoopMiddleware

In [14]:
agent_with_middleware = create_agent(llm, 
    tools=tools, 
    system_prompt = system_prompt.text, 
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "process_time_off": {
                    "allowed_decisions": ["approve", "reject"],
                },
            }
        ),
    ],
    )

In [15]:
user_message = "I want to take leave on 18-12-2025"
config = {'configurable': {'thread_id': user_id}}

In [16]:
agent_with_middleware.invoke({"messages": [{"role": "user", "content": user_message}]}, config=config)

{'messages': [HumanMessage(content='I want to take leave on 18-12-2025', additional_kwargs={}, response_metadata={}, id='daa87b56-6071-4941-b255-5883aab7e071'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 351, 'total_tokens': 368, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e1dc348028', 'id': 'chatcmpl-CniyfSrVNGRtisDKCODKVYuxgmDId', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f372138d-5167-4cd2-8c06-c0ba796c6d1a-0', tool_calls=[{'name': 'get_time_off_balance', 'args': {'user_id': '1'}, 'id': 'call_zaRF0AS9LoxXP2QGjRsYI0aB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 351, 'output_tokens': 17

In [17]:
agent_with_middleware.invoke(Command(resume={"decisions": [{"type": "approve"}]}), config=config)

{'messages': [HumanMessage(content='I want to take leave on 18-12-2025', additional_kwargs={}, response_metadata={}, id='daa87b56-6071-4941-b255-5883aab7e071'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 351, 'total_tokens': 368, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e1dc348028', 'id': 'chatcmpl-CniyfSrVNGRtisDKCODKVYuxgmDId', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f372138d-5167-4cd2-8c06-c0ba796c6d1a-0', tool_calls=[{'name': 'get_time_off_balance', 'args': {'user_id': '1'}, 'id': 'call_zaRF0AS9LoxXP2QGjRsYI0aB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 351, 'output_tokens': 17